In [1]:
from langchain.graphs import Neo4jGraph
import os
import glob
import json
from pathlib import Path
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts.prompt import PromptTemplate

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

/Users/nadcharin/.local/share/virtualenvs/harry-potter-demo-M6hG0Rsr/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Graph model

<img src="img/graph_model.png" width="800"/>

In [8]:
def cypher_chain(question):
    CYPHER_GENERATION_TEMPLATE = """You are an expert Neo4j Cypher translator who understands the question in english and convert to Cypher strictly based on the Neo4j Schema provided and following the instructions below:
    1. Generate Cypher query compatible ONLY for Neo4j Version 5
    2. DO NOT use EXISTS, SIZE keywords in the cypher. Use alias when using the WITH keyword
    3. Use only Nodes and relationships mentioned in the schema
    4. Always enclose the Cypher output inside 3 backticks
    5. ALWAYS do a case-insensitive and fuzzy search for any properties related search. Eg: to search for a Movie name use `toLower(m.title) contains toLower('neo4j')`
    6. Always use aliases to refer the node in the query
    7. Cypher is NOT SQL. So, do not mix and match the syntaxes
    Schema:
    {schema}

    Note: Do not include any explanations or apologies in your responses.
    Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
    Do not include any text except the generated Cypher statement.

    The question is:
    {question}"""
    CYPHER_GENERATION_PROMPT = PromptTemplate(
        input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE)
    graph = Neo4jGraph(url=os.getenv('NEO4J_URI'), username=os.getenv('NEO4J_USER'), password=os.getenv('NEO4J_PASSWORD'),database=os.getenv('NEO4J_DATABASE'))
    cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm = ChatOpenAI(temperature=0, model_name=os.getenv('OPENAI_MODEL')),
    qa_llm = ChatOpenAI(temperature=0,model_name=os.getenv('OPENAI_MODEL')), graph=graph, verbose=True,cypher_prompt=CYPHER_GENERATION_PROMPT)
    return cypher_chain.run(question)


In [9]:
print(cypher_chain("Which actors appeared in all eight Harry Potter films, and who are their characters?"))



> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (c:Cast)-[r:ACTED_IN]->(m:Movie)
WHERE toLower(m.title) CONTAINS toLower('harry potter')
WITH c, r.role AS character, COUNT(m) AS moviesCount
WHERE moviesCount = 8
RETURN c.name, COLLECT(character) AS characters

Full Context:
[{'c.name': 'Tom Felton', 'characters': ['Draco Malfoy']}, {'c.name': 'Bonnie Wright', 'characters': ['Ginny Weasley']}, {'c.name': 'Emma Watson', 'characters': ['Hermione Granger']}, {'c.name': 'Rupert Grint', 'characters': ['Ron Weasley']}, {'c.name': 'Daniel Radcliffe', 'characters': ['Harry Potter']}]

> Finished chain.
The actors who appeared in all eight Harry Potter films and their characters are:

- Tom Felton as Draco Malfoy
- Bonnie Wright as Ginny Weasley
- Emma Watson as Hermione Granger
- Rupert Grint as Ron Weasley
- Daniel Radcliffe as Harry Potter


In [9]:
print(cypher_chain("What year was 'Harry Potter and the Philosopher's Stone' released, and which actors played the roles of Harry Potter, Ron Weasley, and Hermione Granger?"))



> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (m:Movie)
WHERE toLower(m.title) CONTAINS 'harry potter and the philosopher\'s stone'
WITH m
MATCH (c:Cast)-[r:ACTED_IN]->(m)
WHERE toLower(r.role) IN ['harry potter', 'ron weasley', 'hermione granger']
RETURN m.releaseYear, c.name AS Actor, r.role AS Role

Full Context:
[{'m.releaseYear': 2001, 'Actor': 'Emma Watson', 'Role': 'Hermione Granger'}, {'m.releaseYear': 2001, 'Actor': 'Rupert Grint', 'Role': 'Ron Weasley'}, {'m.releaseYear': 2001, 'Actor': 'Daniel Radcliffe', 'Role': 'Harry Potter'}]

> Finished chain.
'Harry Potter and the Philosopher's Stone' was released in 2001. In this film, Daniel Radcliffe played the role of Harry Potter, Rupert Grint portrayed Ron Weasley, and Emma Watson took on the character of Hermione Granger.


In [10]:
print(cypher_chain("How does Harry Potter discover he is a wizard?"))



> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (m:Movie)-[:HAS_ARTICLE]->(a:Article)-[:HAS_CONTENT]->(p:Parent)
WHERE toLower(m.title) CONTAINS 'harry potter' AND toLower(p.text) CONTAINS 'wizard'
RETURN m.title AS MovieTitle, p.text AS DiscoveryText

Full Context:
[{'MovieTitle': 'Harry Potter and the Deathly Hallows – Part 1', 'DiscoveryText': " and former employee at Gringotts Bank. Davis replaced Verne Troyer, who portrayed the character physically in the first film, though Davis had dubbed Griphook's lines.\nTom Felton as Draco Malfoy: a Death Eater and son of Lucius and Narcissa Malfoy.\nRalph Fiennes as Lord Voldemort: An evil power-hungry wizard, and the leader of the Death Eaters.\nMichael Gambon as Professor Albus Dumbledore: Former headmaster of Hogwarts killed by Severus Snape in the previous film.\nBrendan Gleeson as Alastor 'Mad-Eye' Moody: A member of the Order of the Phoenix.\nJohn Hurt as Garrick Ollivander: A wandmaker abducted by the Death Eate